In [1]:
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

加载试验数据

In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        dataset = pickle.load(f, encoding='latin1')
        X = dataset['data']
        X = np.reshape(X, (10000, 3, 32, 32)).transpose(0, 2, 3, 1).astype('float')
        Y = dataset['labels']
        Y = np.array(Y)
    return X, Y
xs = []
ys = []
path = '../cs231n/datasets/cifar-10-batches-py'
for i in range(1, 6):
    filename = os.path.join(path, 'data_batch_%d' % (i, ))
    X, Y = load_data(filename)
    xs.append(X)
    ys.append(Y)
Xtr = np.concatenate(xs)
Ytr = np.concatenate(ys)
X_test, Y_test = load_data(os.path.join(path, 'test_batch'))
print(Xtr.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


画出训练集图像

In [3]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
def plot(Xtr, Ytr, classes): 
    num_classes = len(classes)
    num_colum = 7
    for i in range(num_classes):
        img_index = np.where(Ytr == i)[0]
        for j in range(num_colum):
             index = j * num_classes + i
             plt.subplot(num_colum, num_classes, index + 1)
             plt.imshow(Xtr[img_index[j]].astype('uint'))
             plt.axis('off')
             if j == 0:
                 plt.title(classes[i])
    plt.show()
#plot(Xtr, Ytr, classes)

对数据进行处理

In [4]:
num_training=49000
num_validation=1000
num_test=1000
num_dev=500

#取原数据集的子集以减少训练的时间
X_train = Xtr[:num_training]
Y_train = Ytr[:num_training]
X_val = Xtr[num_training:num_training + num_validation]
Y_val = Ytr[num_training:num_training + num_validation]
X_test = X_test[:num_test]
Y_test = Y_test[:num_test]
index_dev = np.random.choice(num_training, num_dev)
X_dev = Xtr[index_dev]
Y_dev = Ytr[index_dev]

#将数据集reshape
X_train = np.reshape(X_train, (num_training, -1))
X_val = np.reshape(X_val, (num_validation, -1))
X_test = np.reshape(X_test, (num_test, -1))
X_dev = np.reshape(X_dev, (num_dev, -1))

#将数据集Normalize
mean_image = np.mean(X_train, axis= 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

#给数据集加上一个bias
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

编写loss函数

In [5]:
from classifiers import SoftMax
W = np.random.randn(3073, 10) * 0.0001
my, grad = SoftMax.softmax_loss_vectorized(W, X_dev, Y_dev, 1)

[[0.10392132 0.10364837 0.10363999 ... 0.10663733 0.13380481 0.10681231]
 [0.08927846 0.12177259 0.11192778 ... 0.10719449 0.08689706 0.08498017]
 [0.05241907 0.2015891  0.06678168 ... 0.1077399  0.04586803 0.09490373]
 ...
 [0.10400481 0.09205628 0.05588562 ... 0.08486729 0.12630905 0.12455123]
 [0.1565277  0.07571353 0.12498865 ... 0.11210731 0.0699746  0.05154968]
 [0.07666392 0.13248185 0.09754076 ... 0.07272979 0.07801347 0.11562855]]


进行梯度检验

In [ ]:
import Gradient_Check

f = lambda W: SoftMax.softmax_loss_vectorized(W, X_dev, Y_dev, 1)[0]
Gradient_Check.GradientCheck(f, W, grad)

[[0.10391757 0.10364464 0.10367228 ... 0.10663349 0.13379998 0.10680846]
 [0.08928328 0.12177916 0.11187984 ... 0.10720028 0.08690175 0.08498475]
 [0.05242002 0.20159277 0.0667647  ... 0.10774186 0.04586887 0.09490546]
 ...
 [0.10400401 0.09205557 0.05589288 ... 0.08486664 0.12630807 0.12455028]
 [0.15653987 0.07571942 0.1249206  ... 0.11211603 0.06998004 0.05155369]
 [0.07666745 0.13248796 0.09749918 ... 0.07273314 0.07801707 0.11563387]]
[[0.10392506 0.10365211 0.1036077  ... 0.10664117 0.13380962 0.10681616]
 [0.08927364 0.12176601 0.11197574 ... 0.1071887  0.08689237 0.08497558]
 [0.05241811 0.20158543 0.06679866 ... 0.10773794 0.0458672  0.094902  ]
 ...
 [0.10400561 0.09205698 0.05587836 ... 0.08486794 0.12631002 0.12455219]
 [0.15651552 0.07570764 0.12505674 ... 0.11209859 0.06996916 0.05154567]
 [0.07666039 0.13247575 0.09758235 ... 0.07272644 0.07800988 0.11562322]]
compute_grad=1.735865 analyse_grad=1.735865 norm=4.259660e-08
[[0.10392019 0.10364725 0.10363887 ... 0.10663618 

用交叉验证得到较好的reg和learning rate

In [ ]:
from classifiers.Linear_Classifier import LinearSoftMax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]
for lr in np.linspace(learning_rates[0], learning_rates[1], 3):
    for reg in np.linspace(regularization_strengths[0], regularization_strengths[1], 3):
        temp_softmax = LinearSoftMax()
        temp_loss = temp_softmax.train(X_train, Y_train, reg= reg, iteration=1500, learning_rate= lr)
        temp_pre_train = temp_softmax.predict(X_train)
        temp_accuracy_train = np.mean(Y_train == temp_pre_train)
        temp_pre_val = temp_softmax.predict(X_val)
        temp_accuracy_val = np.mean(Y_val == temp_pre_val)
        results[(lr, reg)] = (temp_accuracy_train, temp_accuracy_val)
        if temp_accuracy_train > best_val:
            best_val = temp_accuracy_train
            best_softmax = temp_softmax
            
for key in sorted(results):
    print('reg=%e  lr=%e  train_accuracy=%f, val_accuracy=%f' % 
          (key[0], key[1], results[key][0], results[key][1],))
    

[[1.29977972e-02 8.72712243e-04 6.89079725e-02 ... 2.65233235e-03
  8.69026218e-01 2.84983132e-02]
 [3.99047624e-03 2.39225180e-05 5.88956523e-04 ... 1.31330938e-05
  4.61247148e-06 2.96568160e-03]
 [7.45095507e-02 4.88720947e-04 2.04233127e-06 ... 1.34508827e-05
  3.52458292e-06 4.18104050e-03]
 ...
 [6.82968762e-03 4.36422278e-03 5.20709249e-03 ... 2.61138900e-03
  3.13407909e-04 8.41567841e-01]
 [1.33908636e-02 7.87429192e-03 1.26412474e-02 ... 1.03170232e-01
  5.57179096e-01 4.28359169e-03]
 [2.01525056e-02 1.72388687e-03 3.09755769e-02 ... 3.78932308e-02
  1.35049565e-02 2.18631511e-02]]
[[6.66025544e-01 3.33487001e-01 2.10858700e-04 ... 1.31602660e-06
  5.44824973e-07 1.95242979e-04]
 [8.82189635e-04 5.96348693e-02 5.47655014e-03 ... 4.05051841e-02
  8.40793455e-01 1.21753215e-03]
 [8.68928316e-05 1.41336929e-05 2.99205980e-04 ... 1.17049585e-01
  4.12085048e-03 6.20076892e-03]
 ...
 [5.12461258e-02 1.53162035e-01 1.67753974e-02 ... 5.80961561e-02
  2.77386700e-01 1.39396948e-01]

在测试集上验证结果

In [ ]:
pre_test = best_softmax.predict(X_test)
test_accuracy = np.mean(pre_test == Y_test)
print('在测试集上的准确率为：%f' % (test_accuracy, ))

给W画出图像做一个直观的展示

In [ ]:
W_best = best_softmax.W[:-1, :]
W_max = np.max(W_best)
W_min = np.min(W_best)
W_best = 255 * (W_best - W_min) / (W_max - W_min)
W_best = np.reshape(W_best.T, (10,32,32,3))

for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(W_best[i].astype('uint8'))
    plt.title(classes[i])
    plt.axis('off')
plt.show()

给分类结果做一个直观的图像展示

In [ ]:
num_pic = 16
size = int(np.sqrt(num_pic))
random_index = np.random.choice(range(X_test.shape[0]), num_pic)
pic_selected = X_test[random_index][:, :-1]
pic_max = np.max(pic_selected)
pic_min = np.min(pic_selected)
pic_selected = 255 * (pic_selected - pic_min) / (pic_max - pic_min)
pic_selected = np.reshape(pic_selected, (num_pic, 32, 32, 3))
for i in range(size):
    for j in range(size):
        index = size * i + j
        plt.subplot(size, size, index + 1)
        plt.imshow(pic_selected[index].astype('uint8'))
        plt.title('predict:%s, ture:%s' % (classes[pre_test[random_index[index]]], classes[Y_test[random_index[index]]]))
        plt.axis('off')